In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [3]:
data = pd.read_excel('../../data/data_ford_price.xlsx') 

#  Отбор признаков: мотивация

## Предобработка данных

In [4]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [5]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

In [6]:
x.drop('lat', axis = 1, inplace = True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [8]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

## Метод рекурсивного исключения признаков

In [9]:
from sklearn.feature_selection import RFE

In [10]:
y = data['price']
x = data.drop(columns='price')

In [11]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [12]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
list_RFE=list(selector.get_feature_names_out())
list_RFE

['year', 'cylinders', 'lat']

In [13]:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [14]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [15]:
from sklearn.feature_selection import SelectKBest, f_regression

In [16]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
list_SKB=list(selector.get_feature_names_out())
list_SKB

['year', 'cylinders', 'odometer']

In [17]:
model = LinearRegression()

model.fit(X_train[list_RFE], y_train)
y_predicted = model.predict(X_test[list_RFE])
print('MAE после RFE:        %.3f' % mean_absolute_error(y_test, y_predicted))

model.fit(X_train[list_SKB], y_train)
y_predicted = model.predict(X_test[list_SKB]) 
print('MAE после SelectKBes: %.3f' % mean_absolute_error(y_test, y_predicted))

MAE после RFE:        5096.570
MAE после SelectKBes: 4708.946


В данной задаче метод отбора признаков 'SelectKBest' продемонстрировал более высокую точность на тестовой выборке по сравнению с методом 'RFE'. Тем не менее, необходимо отметить, что результаты могут зависеть от множества факторов, включая особенности данных, выбор метрики для оценки качества модели и настройки параметров алгоритмов.

Я также провёл эксперимент, удалив вручную признак «lat», который был сильно коррелирован с признаком «weather». В результате метод «RFE» отобрал следующие признаки: ['year', 'cylinders', 'weather'] и показал MAE после RFE: 5113.445. В то же время метод «SelectKBest» сохранил свои признаки: ['year', 'cylinders', 'odometer'] и дал MAE после SelectKBest: 4708.946.

Таким образом, я убедился, что «SelectKBest» выбирает признаки более качественно.

## <p align=center> Что посоветовала нейросеть.

Если вы хотите быть уверены в полученных результатах, рекомендуется провести более детальное исследование. Попробуйте изменить параметры методов отбора признаков и метрики оценки качества модели. Также можно попробовать использовать другие методы отбора признаков, например, Mutual Information или Information Gain.

Сравните результаты, полученные с помощью разных методов, и определите, какой из них лучше всего подходит для вашей задачи. Это поможет вам сделать более обоснованный вывод о том, какой метод отбора признаков является наиболее эффективным
